In [8]:
# load the vector files
import sys
import io
setting = "d2v"

vectorFilesDir = "../Data/vectors/"+setting+"/"+setting+".txt"
allPaperVectors = []

with open(vectorFilesDir, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split(" ")
        paper_Vectors = read_data
        allPaperVectors.append(paper_Vectors)
f.close()
        
print("Total vector records:",len(allPaperVectors))
print(allPaperVectors[0])

Total vector records: 3149075
['3', '-0.07245799', '-0.15048164', '-0.04320673', '0.01244448', '0.05051953', '-0.05573996', '0.03158288', '-0.04663554', '-0.00442508', '-0.02417533', '-0.03292065', '0.03798062', '0.08195730', '-0.09100581', '-0.04666801', '-0.06315092', '-0.05957321', '0.09766518', '0.01981102', '0.09956500', '-0.02059892', '-0.02321497', '0.10300557', '0.09654117', '0.02085607', '0.15179265', '0.03320639', '0.04716884', '0.04259005', '-0.01022485', '0.07371941', '0.02970656', '0.18967280', '0.07049462', '-0.07849123', '0.10272161', '0.05396378', '0.04138396', '0.08093689', '-0.04713648', '-0.08277001', '0.06004119', '0.15147503', '-0.10719796', '-0.06268646', '0.15823838', '0.10273122', '0.04453533', '-0.00394740', '-0.01239040', '-0.06826647', '-0.02995823', '0.14925463', '0.12254845', '-0.05894163', '0.11628735', '0.03898517', '0.01221054', '-0.00804257', '-0.06178775', '-0.04752085', '-0.04040224', '0.09192738', '0.01171173', '0.02951661', '-0.02156392', '-0.024588

In [9]:
import re
import numpy as np
import os

# collect data
fileDir = "../Data/filteredSameNameAuthor/filter=10/"
fileList = os.listdir(fileDir)
fileList.sort()
print(fileList)


['alfredo martinez.txt', 'alfredo martinez0.txt', 'alfredo martinez1.txt', 'amit patel.txt', 'amit patel0.txt', 'amit patel1.txt', 'ana castro.txt', 'ana castro0.txt', 'ana castro1.txt', 'ana castro2.txt', 'anna ferrari.txt', 'anna ferrari0.txt', 'anna ferrari1.txt', 'bin liu.txt', 'bin liu0.txt', 'bin liu1.txt', 'carmen moreno.txt', 'carmen moreno0.txt', 'carmen moreno1.txt', 'carmen torres.txt', 'carmen torres0.txt', 'carmen torres1.txt', 'chao liu.txt', 'chao liu0.txt', 'chao liu1.txt', 'cheng luo.txt', 'cheng luo0.txt', 'cheng luo1.txt', 'chung-may yang.txt', 'chung-may yang0.txt', 'chung-may yang1.txt', 'david g lloyd.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'fang liu.txt', 'fang liu0.txt', 'fang liu1.txt', 'feng liu.txt', 'feng liu0.txt', 'feng liu1.txt', 'feng xu.txt', 'feng xu0.txt', 'feng xu1.txt', 'francisco esteves.txt', 'francisco esteves0.txt', 'francisco esteves1.txt', 'francisco j blanco.txt', 'francisco j blanco0.txt', 'francisco j blanco1.txt', 'giovanni volpe

In [10]:
# remove author(positive sample) from other(negative sample)
import random
def extractNegativeSample(positiveSample, allSample):
    negativeSample = [x for x in allSample if x not in positiveSample]
    print("Total negative sample size:", len(negativeSample))
    return negativeSample

In [11]:
# collect class vectors
import pandas as pd
import numpy as np

def extractVectors(author_pids, NegativeSample_pid, allPaperVectors):
    # extract class one vectors
    author_features = []
    for pid in author_pids:
         for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                author_features.append(paper_Vectors)
    print("Positive sample size: ", len(author_features))
    classOne = pd.DataFrame(author_features)
    classOne["label"] = 0
    # extract class two vectors
    other_features = []
    for pid in NegativeSample_pid:
        for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                other_features.append(paper_Vectors)
    print("Negative sample size: ", len(other_features))
    classTwo = pd.DataFrame(other_features)
    classTwo["label"] = 1
    return classOne, classTwo


In [12]:
# combine data from different class get all data
def combineClassesData(classOne,classTwo):
    combinedData = pd.concat([classOne, classTwo])
    combinedData = combinedData.sample(frac=1).reset_index(drop=True)
    # take the paper id out
    paperID = combinedData[0]
    # split data and label
    data = combinedData.drop([0,'label'], axis=1)
    label = combinedData['label']
    print("Total sample size and shape: ",data.shape)
    return data, label, paperID

In [13]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score,accuracy_score)
# cross validation
def k_fold_cv(author, data, label, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for counter,(train_index, test_index) in enumerate(kf.split(data)):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        allTrueLabel.extend(test_true_label)
        allPredLabel.extend(label_pred)
#         # get predict proba
#         proba = classifier.predict_proba(data_test)
        # find out which sample cause the issue
        print("Pred: ",label_pred)
        print("True: ", test_true_label.values.tolist())
        print("Mislabeled sample: ",end='')
        for i in range(len(test_true_label)):
            if(label_pred[i]!=test_true_label[test_index[i]]):
                print(paperID[test_index[i]]+",",end='')
        print()
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='binary')
    precision = precision_score(allTrueLabel, allPredLabel)
    recall = recall_score(allTrueLabel, allPredLabel)
    tn,fp,fn,tp = metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel()
    
    print("Author: ", author)
    print("Classifier: ",classifier)
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    print("Accuracy: ",accuracy)
    print("F1: ", f1)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return accuracy, f1, precision, recall, tn, fp, fn, tp
    # return ppv, npv, specificity, sensitivity, accuracy, f1proba = linear_svc.predict_proba(allDatas)

In [14]:
from sklearn import svm
from sklearn import linear_model
from sklearn.decomposition import PCA

# loop through files in directory |
# add name to name list
# author as positive sample, other as all samples
name_list = []
# create name list for all authors have same name
for file in fileList:
    if not file.startswith('.'):
        if not re.match(r'\D*\d+.txt$', file):
            # fix the coding issue
            name_list.append(file.encode("utf-8", "surrogateescape").decode('utf8','surrogateescape')[:-4])
# print(name_list)

# loop through all the author and gather result
allauthor = []
authorSampleSize = []
allSampleSize = []
allaccuracy = []
allf1 = []
allprecision = []
allrecall = []
alltn = []
allfp = []
allfn = []
alltp = []

for name in name_list:
    other_pids = []
    # read other sample
    with open((fileDir+name+".txt").encode('utf-8'), 'r', encoding = 'utf8') as f:
        for line in f:
            other_pids.extend(line.strip().split(" "))
#     print(name)
    for file in fileList:
        file=file.encode("utf-8", "surrogateescape").decode('utf8','surrogateescape')
        if not file.startswith('.'):
            if re.match(r'\D*\d+.txt$', file):
                if name in file:
                    print(os.path.splitext(file)[0])
                    # add author to list for final output
                    allauthor.append(os.path.splitext(file)[0])
                    author_pids = []
                    # read author sample
                    with open((fileDir+os.path.splitext(file)[0]+".txt").encode('utf-8'), 'r', encoding = 'utf8') as f:
                        for line in f:
                            author_pids.extend(line.strip().split(" "))
                    # print properties
                    authorSampleSize.append(len(author_pids))
                    allSampleSize.append(len(other_pids))
                    print(len(author_pids))
                    print(len(other_pids))
                    # remove author(positive sample) from other(all sample) to create negative sample
                    NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
                    print(len(NegativeSample_pid))
                    # collect all vector
                    classOne, classTwo = extractVectors(author_pids,NegativeSample_pid,allPaperVectors)
                    print(classOne.shape)
                    print(classTwo.shape)
                    # combine data from different class get all data
                    data, label, paperID = combineClassesData(classOne, classTwo)
#                     # PCA transform
#                     pca = PCA(n_components=2)
#                     pca_transformed = pd.DataFrame(pca.fit_transform(X=data, y=label))
#                     pca_transformed["label"] = label
#                     # 10 fold cv SVM linear kernel
#                     linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True,cache_size=4000)
#                     accuracy, f1, precision, recall, tn, fp, fn, tp= k_fold_cv(os.path.splitext(file)[0],data, label, linear_svc,10)
#                     # 10 fold cv SVM rbf kernel
#                     rbf_svc = svm.SVC(kernel='rbf')
#                     accuracy, f1, precision, recall, tn, fp, fn, tp = k_fold_cv(os.path.splitext(file)[0],data, label, rbf_svc,10)
                    # 10 fold cv logistic regression
                    logistic = linear_model.LogisticRegression(n_jobs=-1)
                    accuracy, f1, precision, recall, tn, fp, fn, tp = k_fold_cv(os.path.splitext(file)[0],data, label, logistic,10)
                    allaccuracy.append(accuracy)
                    allf1.append(f1)
                    allprecision.append(precision)
                    allrecall.append(recall)
                    alltn.append(tn)
                    alltp.append(tp)
                    allfn.append(fn)
                    allfp.append(fp)
# write evaluation result to excel
output = pd.DataFrame({'author':allauthor,"AuthorSampleSize":authorSampleSize,
                       "accuracy":allaccuracy,"f1":allf1, "precision":allprecision,
                      "recall":allrecall, "AllSameNameSampleCount":allSampleSize,
                      "True positive": alltp, "True negative":alltn,
                      "False positive": allfp, "False negative": allfn})
filename = "logistic_regression"+setting+"_filter=10.csv"
output.to_csv("../result/"+filename, encoding='utf-8',index=False)
print("Done")

alfredo martinez0
17
37
Total negative sample size: 20
20
Positive sample size:  17
Negative sample size:  20
(17, 102)
(20, 102)
Total sample size and shape:  (37, 100)
Pred:  [1 1 0 1]
True:  [1, 1, 0, 1]
Mislabeled sample: 
Pred:  [0 0 1 1]
True:  [0, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 1 1]
True:  [1, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 0 0]
True:  [1, 0, 0, 0]
Mislabeled sample: 
Pred:  [1 0 0 1]
True:  [1, 0, 0, 1]
Mislabeled sample: 
Pred:  [1 1 1 0]
True:  [1, 1, 1, 0]
Mislabeled sample: 
Pred:  [1 1 1 0]
True:  [1, 1, 1, 0]
Mislabeled sample: 
Pred:  [0 1 0]
True:  [0, 1, 0]
Mislabeled sample: 
Pred:  [0 0 1]
True:  [0, 0, 1]
Mislabeled sample: 
Pred:  [0 0 1]
True:  [0, 0, 1]
Mislabeled sample: 
Author:  alfredo martinez0
Classifier:  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
        

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


 1.0
Recall:  1.0
alfredo martinez1
20
37
Total negative sample size: 17
17
Positive sample size:  20
Negative sample size:  17
(20, 102)
(17, 102)
Total sample size and shape:  (37, 100)
Pred:  [1 0 1 0]
True:  [1, 0, 1, 0]
Mislabeled sample: 
Pred:  [1 1 0 1]
True:  [1, 1, 0, 1]
Mislabeled sample: 
Pred:  [1 1 0 0]
True:  [1, 1, 0, 0]
Mislabeled sample: 
Pred:  [0 1 0 1]
True:  [0, 1, 0, 1]
Mislabeled sample: 
Pred:  [0 1 1 1]
True:  [0, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 0]
True:  [1, 1, 1, 0]
Mislabeled sample: 
Pred:  [0 1 0 1]
True:  [0, 1, 0, 1]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Author:  alfredo martinez1
Classifier:  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', to

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Positive sample size:  11
Negative sample size:  18
(11, 102)
(18, 102)
Total sample size and shape:  (29, 100)
Pred:  [0 0 1]
True:  [0, 0, 1]
Mislabeled sample: 
Pred:  [1 1 1]
True:  [1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 0]
True:  [1, 1, 0]
Mislabeled sample: 
Pred:  [0 1 1]
True:  [0, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1]
True:  [1, 0, 1]
Mislabeled sample: 23154265,
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [1 0 0]
True:  [1, 0, 1]
Mislabeled sample: 24758713,
Pred:  [0 0 1]
True:  [0, 0, 1]
Mislabeled sample: 
Pred:  [1 1 1]
True:  [1, 1, 1]
Mislabeled sample: 
Pred:  [1 1]
True:  [1, 1]
Mislabeled sample: 
Author:  amit patel0
Classifier:  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

 

...

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Positive sample size:  13
Negative sample size:  15
(13, 102)
(15, 102)
Total sample size and shape:  (28, 100)
Pred:  [1 1 0]
True:  [1, 1, 0]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [1 0 1]
True:  [1, 0, 1]
Mislabeled sample: 
Pred:  [1 0 1]
True:  [1, 0, 1]
Mislabeled sample: 
Pred:  [0 1 1]
True:  [0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 1]
True:  [1, 0, 1]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [1 1 1]
True:  [1, 1, 1]
Mislabeled sample: 
Pred:  [1 0]
True:  [1, 0]
Mislabeled sample: 
Pred:  [1 1]
True:  [0, 1]
Mislabeled sample: 25331363,
Author:  yu-jun zhao0
Classifier:  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0   

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Positive sample size:  15
Negative sample size:  13
(15, 102)
(13, 102)
Total sample size and shape:  (28, 100)
Pred:  [0 1 1]
True:  [1, 1, 1]
Mislabeled sample: 25331363,
Pred:  [0 0 1]
True:  [0, 0, 1]
Mislabeled sample: 
Pred:  [1 1 0]
True:  [1, 1, 0]
Mislabeled sample: 
Pred:  [1 1 1]
True:  [1, 1, 1]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [1 0 1]
True:  [1, 0, 1]
Mislabeled sample: 
Pred:  [0 0 0]
True:  [0, 0, 0]
Mislabeled sample: 
Pred:  [0 0 1]
True:  [0, 0, 1]
Mislabeled sample: 
Pred:  [0 0]
True:  [0, 0]
Mislabeled sample: 
Pred:  [0 1]
True:  [0, 1]
Mislabeled sample: 
Author:  yu-jun zhao1
Classifier:  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0   

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [7]:
# hard code to read the file one by one
# store the features for classification
author_pids = []
other_pids = []
name = "michael wagner0"
# author as positive sample, other as all samples
with open(fileDir+name+".txt", 'r', encoding = 'utf8') as f:
    for line in f:
        author_pids.extend(line.strip().split(" "))

with open(fileDir+"michael wagner.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        other_pids.extend(line.strip().split(" "))
        
# size of each class
print(len(author_pids))
print(len(other_pids))

16
255


In [8]:
# extract negative Sample
NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
print("Choicen negative sample ", len(NegativeSample_pid))

Total negative sample size: 239
Choicen negative sample  239


In [9]:
classOne, classTwo = extractVectors(author_pids,NegativeSample_pid,allPaperVectors)
print(classOne.shape)
print(classTwo.shape)

Positive sample size:  16
Negative sample size:  239
(16, 102)
(239, 102)
16


In [43]:
data, label, paperID = combineClassesData(classOne, classTwo)

Total sample size and shape:  (255, 100)


In [48]:
from sklearn import svm
# create linear SVM model
linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True)

# fit model and do 10-fold cv
k_fold_cv(name,data, label, linear_svc,10)

# get number of support vectors for each class
print(linear_svc.n_support_)

'''
# compute the distance to decision boundry (Not same as confidence measure)
Distance = linear_svc.decision_function(allDatas)

# computer the confidence measure (Platt scaling: transforming the outputs of a 
# classification model into a probability distribution over classes)
# P(class/input) = 1 / (1 + exp(A * f(input) + B))
# P(class/input) is the probability that “input” belongs to “class” 
# and f(input) is the signed distance of the input datapoint from the boundary,
# which is basically the output of “decision_function”. 

proba = linear_svc.predict_proba(allDatas)

'''

# create rbf SVM model with C=10 where (C*Error) is added into minimize function
# C big means error matter more
rbf_svc = svm.SVC(kernel='rbf',probability=True)

# fit model and do 10-fold cv
k_fold_cv(name,data, label, rbf_svc)

# get number of support vectors for each class
print(rbf_svc.n_support_)


Pred:  [1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
True:  [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
True:  [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
Mislabeled sample: 20459663,
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 26330430,18333798,25493568,
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 27054571,
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.9372549019607843, 0.90689846788918, 0.9372549019607843, 1.0, 0, 16, 0, 239)